In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing packages

In [289]:
import pandas as pd
import networkx as nx

In [54]:
# !ls "/content/drive/MyDrive/Network Science Project Files"


In [55]:
file_path = "/content/drive/MyDrive/Network Science Project Files"

In [275]:
df = pd.read_csv(file_path+'/all_records.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (20,21,22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [276]:
# df.head()

In [277]:
df.columns = list(map(str.strip, df.columns))

In [278]:
df['NORMED?'] = df['NORMED?'].str.strip()
df['CUE'] = df['CUE'].str.strip()
df['TARGET'] = df['TARGET'].str.strip()

In [279]:
df = df[df['NORMED?']=='YES']

In [280]:
df = df[['CUE','TARGET','NORMED?','#G','#P','FSG','BSG','QPS','TPS']]

* CUE	Normed Word
* TARGET	Response to Normed Word
* NORMED?	Is Response Normed?
* \#G	Group size
* \#P	Number of Participants Producing Response
* FSG	Forward Cue-to-Target Strength
* BSG	Backward Target-to-Cue Strength
* QPS	Cue: Part of Speech
* TPS	Target: Part of Speech





In [281]:
## filtering POS for CUES and TARGETS to be, nouns verbs and adjectives
exclude_list = ['PP','AV','ADV','P','I','C']
df1 = df[~df['QPS'].str.contains('|'.join(exclude_list))]
df1['TPS'] = df1['TPS'].dropna()
df1 = df1[df1['TPS']!=' ']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


* N	Noun
* V	Verb
* AJ	Adjective
* AD	Adverb
* P	Pronoun
* PP	Preposition
* I	Interjection
* C	Conjunction

In [282]:
df1['BSG'] = df1['BSG'].astype(float)

In [283]:
df1_n_bsg = df1[df1['BSG']==0]
df1_n_bsg = df1_n_bsg[df1_n_bsg['#P']>2]

In [327]:
# df1_n_bsg_a_b = df1_n_bsg[(df1_n_bsg['CUE'].str.startswith('A')) | (df1_n_bsg['CUE'].str.startswith('B'))]
df1_n_bsg_a = df1_n_bsg[(df1_n_bsg['CUE'].str.startswith('A'))]

In [328]:
lst = list(set(list(df1_n_bsg_a['CUE'])+list(df1_n_bsg_a['TARGET'])))

In [329]:
G = nx.Graph()

In [330]:
for n in lst:
  G.add_node(n)


In [331]:
for row in df1_n_bsg_a.itertuples():
  # print(row.CUE)
  G.add_edge(row.CUE,row.TARGET)

In [333]:
nx.write_gml(G,'cue_target.gml')